In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
#Importing data
data = spark.read.csv('/FileStore/tables/SMSSpamCollection', inferSchema=True, sep='\t')
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [0]:
#Relabel Columns
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import length

In [0]:
data = data.withColumn('length',length(data['text']))
data.show(3)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
+-----+--------------------+------+
only showing top 3 rows



In [0]:
#Checking if there is any difference in length between ham and spam mails
data.groupBy('class').mean('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham| 71.4545266210897|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipeline = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])
cleaner = data_prep_pipeline.fit(data)
cleaned_data = cleaner.transform(data)
cleaned_data.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|          stop_token|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [0]:
cleaned_data = cleaned_data.select('features','label')

In [0]:
train_data, test_data = cleaned_data.randomSplit([0.7,0.3])

In [0]:
spam_detector = nb.fit(train_data)

In [0]:
test_preds = spam_detector.transform(test_data)
test_preds.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13424,[0,1,2,4,3...|  1.0|[-1228.8010211812...|[1.77538311407153...|       1.0|
|(13424,[0,1,2,12,...|  1.0|[-1132.9355834405...|[4.54113257448299...|       1.0|
|(13424,[0,1,2,15,...|  1.0|[-1167.0497755591...|[2.88905320494872...|       1.0|
|(13424,[0,1,7,8,1...|  0.0|[-1159.9979096862...|[1.0,1.5536430660...|       0.0|
|(13424,[0,1,13,27...|  1.0|[-1178.1944096099...|[1.25271076684737...|       1.0|
|(13424,[0,1,14,31...|  0.0|[-216.37197710675...|[1.0,4.1134085822...|       0.0|
|(13424,[0,1,20,27...|  0.0|[-982.44950496027...|[0.99999998778259...|       0.0|
|(13424,[0,1,27,35...|  0.0|[-1470.5760754225...|[0.99999999999931...|       0.0|
|(13424,[0,1,30,11...|  0.0|[-596.34342040097...|[1.0,6.6672773230...|       0.0|
|(13424,[0,1,31,

In [0]:
 from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
accuracy = MulticlassClassificationEvaluator()
acc = accuracy.evaluate(test_preds)
print(acc)

0.9220263466909286
